## Cleaning Marketing Campaign Messy Data
The Dataset is Messy. Column names have 

In [64]:
import pandas as pd
import numpy as np

In [65]:
df = pd.read_csv('marketing_campaign_data_messy.csv')
df.head()

,Campaign_ID,Campaign_Name,Start_Date,End_Date,Channel,Impressions,Clicks,Spend,Conversions,Active,Clicks,Campaign_Tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,0,NaN,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13 00:00:00,2023-12-20,Email,77820,843,1323.39,51.0,No,NaN,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,2023-10-30,2023-11-03,TikTok,55886,2019,2180.38,135.0,True,NaN,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22 00:00:00,2023-04-23,Facebook,7265,169,252.44,30.0,Yes,NaN,FA


## Cleaning Header

Header have blank spaces, that can make it harder to call, So I will remove the spaces, and also convert column header in Lower Case

In [66]:
print(df.columns.tolist())

[' Campaign_ID ', 'Campaign_Name', 'Start_Date', 'End_Date', 'Channel', 'Impressions', 'Clicks ', 'Spend', 'Conversions', 'Active', 'Clicks', 'Campaign_Tag']


In [67]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print(df.columns.tolist()) 

['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


## Type and Currency Cleaning

In [68]:
df[df['spend'].astype(str).str.contains(r'\$')].head()

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,clicks,campaign_tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
21,CMP-00022,Q2_BlackFriday_CMP-00022,2023-12-26 00:00:00,2024-01-02,Google Ads,54707,1735,$2428.4,NaN,Y,NaN,GO
22,CMP-00023,Q4_Winter_CMP-00023,2023-11-08 00:00:00,2023-11-24,Google Ads,86305,2637,$4726.22,216.0,0,86385.0,GO
31,CMP-00032,Q2_Summer_CMP-00032,2023-05-07 00:00:00,2023-05-29,Instagram,66725,1424,$2759.35,251.0,True,NaN,IN
32,CMP-00033,Q2_BlackFriday_CMP-00033,2023-10-27 00:00:00,2023-10-29,Instagram,85654,2087,$2393.02,122.0,0,NaN,IN


In [69]:
# Remove dollar signs and convert 'spend' to numeric
df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]', '', regex=True)
df['spend'] = pd.to_numeric(df['spend'])

## Categorical Typos (Fuzzy Logic)

In [70]:
df['channel'].value_counts()

channel
TikTok        399
Facebook      391
Email         359
Google Ads    346
Instagram     340
E-mail         21
Insta_gram     19
Tik_Tok        16
Gogle          14
Facebok        14
Name: count, dtype: int64

In [71]:
clean_up = {'Facebok' : 'Facebook',
    'Insta_gram' : 'Instagram',
    'Gogle' : 'Google',
    'Tik_Tok' : 'TikTok',
    'E-mail' : 'Email',
    'N/A' : np.nan}

df['channel'] = df['channel'].replace(clean_up)

In [72]:
df['channel'].unique()

array(['TikTok', 'Facebook', 'Email', 'Instagram', 'Google Ads', nan,
       'Google'], dtype=object)

## Handling Mixed Booleans

In [73]:
df['active'].value_counts()

active
1        319
Yes      306
True     297
No       291
False    281
Y        273
0        253
Name: count, dtype: int64

In [74]:
bool_map = {'Yes' : True,
            'No' : False,
            'Y' : True,
            '0' : False,
            1 : True,
            0 : False,
            '1' : True,
            'True' : True,
            'False' : False}

df['active'] = df['active'].replace(bool_map).fillna(False).astype(bool)

/var/folders/z8/nbvk5kgd18g2blb0t3nbxnmm0000gn/T/ipykernel_56246/4289463038.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['active'] = df['active'].replace(bool_map).fillna(False).astype(bool)


In [75]:
df['active'].value_counts()

active
True     1195
False     825
Name: count, dtype: int64

## Parsing Date

How Start and End Date are completely different and badly formatted.

In [76]:
df.head()

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,clicks,campaign_tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,102.82,20.0,True,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,False,NaN,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13 00:00:00,2023-12-20,Email,77820,843,1323.39,51.0,False,NaN,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,2023-10-30,2023-11-03,TikTok,55886,2019,2180.38,135.0,True,NaN,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22 00:00:00,2023-04-23,Facebook,7265,169,252.44,30.0,True,NaN,FA


In [ ]:
print(df['start_date'].dtype)

#Object means data is stored as strings

object


In [78]:
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
df

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,clicks,campaign_tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24,2023-12-13,TikTok,16795,197,102.82,20.0,True,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06,2023-05-12,Facebook,1860,30,24.33,1.0,False,NaN,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13,2023-12-20,Email,77820,843,1323.39,51.0,False,NaN,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,NaT,2023-11-03,TikTok,55886,2019,2180.38,135.0,True,NaN,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22,2023-04-23,Facebook,7265,169,252.44,30.0,True,NaN,FA
...,...,...,...,...,...,...,...,...,...,...,...,...
2015,CMP-00400,Q3_Summer_CMP-00400,2023-10-31,2023-11-13,TikTok,30592,586,503.95,77.0,True,NaN,TI
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01,2023-09-26,Google Ads,20097,897,1641.00,162.0,False,NaN,GO
2017,CMP-01050,Q2_Launch_CMP-01050,2023-02-09,2023-02-21,Instagram,33254,1117,883.82,214.0,False,NaN,IN
2018,CMP-01118,Q4_Winter_CMP-01118,2023-03-30,2023-04-27,Facebook,68728,2960,4198.50,591.0,True,NaN,FA


## Logical Error (More Clicks than Impressions)

In [95]:
#Turns out there are two dupilicate columns named 'clicks'
df = df.loc[:, ~df.columns.duplicated()]

In [96]:
df.query('clicks > impressions') 

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,campaign_tag


## Logical Integrity (Start time cannot be before End Time)

In [103]:
df.loc[df['start_date'] > df['end_date']].head(3)

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,campaign_tag
23,CMP-00024,Q4_Winter_CMP-00024,2023-05-06,2023-05-01,Instagram,29693,995,1793.94,179.0,False,IN
54,CMP-00055,Q1_Launch_CMP-00055,2023-09-01,2023-08-27,Google Ads,18159,471,563.31,33.0,False,GO
71,CMP-00072,Q1_Summer_CMP-00072,2023-02-01,2023-01-27,Email,66697,1340,2090.74,NaN,True,EM


In [105]:
wrong_end_dates = df['start_date'] > df['end_date']
df.loc[wrong_end_dates, 'end_date'] = df.loc[wrong_end_dates, 'start_date'] + pd.Timedelta(days=30)

## String Parsing (Feature Extraction)

In [109]:
df['campaign_name'].head(10)

0         Q4_Summer_CMP-00001
1         Q1_Launch_CMP-00002
2         Q3_Winter_CMP-00003
3    Q1_BlackFriday_CMP-00004
4         Q2_Winter_CMP-00005
5    Q4_BlackFriday_CMP-00006
6         Q3_Launch_CMP-00007
7         Q4_Launch_CMP-00008
8    Q4_BlackFriday_CMP-00009
9         Q2_Winter_CMP-00010
Name: campaign_name, dtype: object

In [112]:
df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_')
df['season']

0            Summer
1            Launch
2            Winter
3       BlackFriday
4            Winter
           ...     
2015         Summer
2016         Summer
2017         Launch
2018         Winter
2019         Launch
Name: season, Length: 2020, dtype: object